# Implementation from Scratch

<br />

I am going to rewrite codes of 3 layers of neural network that I created on the previous sprint to expand them to any structures by using classes.

## [Task 1] Class All Bonding Layers

In [ ]:
class FC():
    """
    All bonding layers from a layer of n_nodes1 to a layer of n_nodes2
    
    Parameters
    ----------
    n_nodes1 : int
        Number of nodes of the previous layer
    
    n_nodes2 : int
        Number of nodes of the following layer
    
    initializer : Instance of initialization method
    
    optimizer : Instance of optimisation method
    """
    
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        
        # Initialize
        self.initializer = SimpleInitializer()
        # Initialize self.W and self.B by using initializer method
        
        
        self.optimizer = optimizer
    
    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """        
        pass
        return A
    
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        pass
        # 更新
        self = self.optimizer.update(self)
        return dZ